# Ch6. Descritor 로 더 멋진 객체 만들기
### Reference
- [DOC](https://docs.python.org/ko/3/howto/descriptor.html)


# 디스크립터 개요

- 디스크립터(Descriptor)란 다음 매직 메서드 중에 최소 한 개 이상을 포함하는 객체
    - `__get__`, `__set__` , `__delete__`, `__set_name__`
- @property의 단점은 속성을 처리하는 메서드가 하위 클래스 사이에서만 공유될 수 있다는 것
    - 서로 관련이 없는 클래스 사이에 같은 (프로퍼티 게터나 세터) 구현을 공유 할 수 없음
- 디스크립터는 **재사용 가능한(반복 가능한) 프로퍼티 로직을 구현**할 때는 물론 다른 여러 용도에도 사용할 수 있음
    - `__get__` 메서드 뒤쪽으로 모든 종류의 논리를 추상화할 수 있음
    - 클라이언트에게 내용을 숨긴 채로 모든 유형의 변환을 투명하게 실행(캡슐화)

- 디스크립터는 데코레이터가 클래스 메서드에서도 동작할 수 있도록 도와 더 나은 데코레이터를 만들 수 있게 해줌
- 클라이언트가 새용하게 되는 내부 API에 대해서는 디스크립터를 사용하는 것이 좋음
    - 라이브러리, 프레임워크에서의 기능 확장
- 비즈니스 로직을 넣으면 안됨

In [2]:
class Attribute:
    value = 42

class Client:
    """디스크립터 프로토콜 사용"""
    attribute = Attribute()

print(Client().attribute)
print(Client().attribute.value)

42


> 디스크립터 객체는 항상 클래스 속성으로 선언해야 한다!   
>  >객체를 인스턴스 속성으로 생성하면 동작하지 않음


In [1]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class DescriptorClass:
    """
    디스크립터로 인식, 
    객체 자체를 반환하는 것이 아니라
    _get__ 매직 메서드의 결과를 반환
    """
    def __get__(self, instance, owner):
        if instance is None:
            return self
        logger.info(
            "Call: %s.__get__(%r, %r)",
            self.__class__.__name__,
            instance,
            owner,
        )
        return instance


class ClientClass:
    """디스크립터 객체는 항상 클래스 속성으로 선언"""
    descriptor = DescriptorClass()

client = ClientClass()


In [2]:
client = ClientClass()

client.descriptor is client

INFO:__main__:Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x000001563C0F46D0>, <class '__main__.ClientClass'>)


True

## 디스크립터 프로토콜의 메서드 탐색
`descr.__get__(self, obj, type=None) -> value`   
`descr.__set__(self, obj, value) -> None`   
`descr.__delete__(self, obj) -> None`   
이것이 전부입니다. 이러한 메서드 중 하나를 정의하십시오,    
그러면 객체를 디스크립터로 간주하고 어트리뷰트로 조회될 때 기본 동작을 재정의할 수 있음

### `__get__(self, instance, owner)`
- self : 디스크립터 객체
- instance : 클라이언트 객체, 디스크립터를 호출할 객체
- owner : 해당 객체의 클래스, 바로 호출하기 위해 사용

In [3]:
class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return f"{self.__class__.__name__}.{owner.__name__}"
        return f"value for {instance}"


class ClientClass:
    descriptor = DescriptorClass()

ClientClass.descriptor

'DescriptorClass.ClientClass'

In [4]:
ClientClass().descriptor

'value for <__main__.ClientClass object at 0x000001563C030280>'

## 디스크립터 유형
### non-data 디스크립터
- get만 구현한 디스크립터
- 객체의 사전보다 후순위로 적용
- 즉, 객체의 사전에 디스크립터와 동일한 이름의 키가 있으면 객체의 사전 값만 적용

### data 디스크립터
- set, delete로 정의한 디스크립터
- [doc](https://docs.python.org/ko/3/howto/descriptor.html#orm-example)
- ORM 
  - 데이터를 외부 데이터베이스에 저장되게 함
  - 파이썬 인스턴스는 데이터베이스 테이블에 대한 키만 보유
  - 디스크립터가 조회, 갱신 처리

# 디스크립터 실전
## 다른 형태의 디스크립터

### 전역 상태 공유(global shared state) 이슈
- 디스크립터는 클래스 속성으로 설정해야함
- 클래스 속성의 문제점은 해당 클래스의 모든 인스턴스에서 공유된다는 것

### 객체의 사전에 접근하기
- `__dcit__` 사전에 값을 설정하고 검색해야 함
> 항상 인스턴스의 `__dict__`속성에서 데이터를 저장하고 반환하라

In [1]:
class SharedDataDescriptor:
    def __init__(self, initial_value):
        self.value = initial_value

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.value

    def __set__(self, instance, value):
        self.value = value


class ClientClass:
    descriptor = SharedDataDescriptor("first value")

In [2]:
client1 = ClientClass()
client2 = ClientClass()

client2.descriptor = "value for client 2"
client2.descriptor
client1.descriptor

'value for client 2'

- 인스턴스(client12)를 바꾸면 다른 인스턴스(clieet1)도 바뀜

#### 약한 참조 사용
- wekref 모듈 => 약한 키 생성
> 주의사항
- 인스턴스 객체는 더 이상 속성을 보유하지 않고 **디스크립터가 속성을 보유**
- 객체는 `__hash__` 메서드를 구현하여 해시가 가능해야 함

In [4]:
from weakref import WeakKeyDictionary


class DescriptorClass:
    def __init__(self, initial_value):
        self.value = initial_value
        self.mapping = WeakKeyDictionary()

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.mapping.get(instance, self.value)

    def __set__(self, instance, value):
        self.mapping[instance] = value


class ClientClass:
    """
    >>> client1 = ClientClass()
    >>> client2 = ClientClass()

    >>> client1.descriptor = "new value"

    client1 must have the new value, whilst client2 has to still be with the
    default one:

    >>> client1.descriptor
    'new value'
    >>> client2.descriptor
    'default value'

    Changing the value for client2 doesn't affect client1

    >>> client2.descriptor = "value for client2"
    >>> client2.descriptor
    'value for client2'
    >>> client2.descriptor != client1.descriptor
    True
    """

    descriptor = DescriptorClass("default value")

In [9]:
client1 = ClientClass()
client2 = ClientClass()

client1.descriptor = "new value"
print(client1.descriptor)
print(client2.descriptor)

client2.descriptor = "value for client2"
client2.descriptor != client1.descriptor

new value
default value


True

# 디스크립터 분석
## 파이썬 내부에서의 활용

### 함수와 메서드
- 함수는 `__get__` 메서드를 구현했기 때문에 클래스 안에서 메서드처럼 동작
- 메서드는 추가 파라미터를 가진 함수
- 즉, 메서드는 객체를 수정하는 또 다른 함수
- 객체 안에서 정의되었기 때문에 객체에 binding 되어 있다고 말함

In [ ]:
class MyClass:
    def method(self, ...):
        self.x = 1

In [ ]:
class MyClass: pass

def method(myclass_instance, ...):
    myclass_instance.x = 1
    
method(MyClass())

In [ ]:
instance = MyClass()
instance.method(...)

instance = MyClass()
MyClass.method(instance, ...)

#### 예제
- 외부에서 호출 가능한 형태의 함수 또는 메서드를 클래스 내에 호출 가능한 객체로 정의하는 예제

In [16]:
from types import MethodType


class Method:
    def __init__(self, name):
        self.name = name

    def __call__(self, instance, arg1, arg2):
        print(f"{self.name}: {instance} called with {arg1} and {arg2}")


class MyClass:
    method = Method("Internal call")

In [17]:
instance = MyClass()
Method('External call')(instance, "first", "second")
instance.method("first", "second")

External call: <__main__.MyClass object at 0x0000029FEDB35C70> called with first and second


TypeError: __call__() missing 1 required positional argument: 'arg2'

- 파라미터의 위치가 한 칸씩 밀린다.

=> 디스크립터로 변경해서 instance.method 호출시 method.__get__ 메서드를 먼저 호출하게 함

In [20]:
class NewMethod:
    """디스크립터로 변경"""
    def __init__(self, name):
        self.name = name

    def __call__(self, instance, arg1, arg2):
        print(f"{self.name}: {instance} called with {arg1} and {arg2}")

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return MethodType(self, instance)


class MyClass1:
    method = NewMethod("Internal call")

In [21]:
instance = MyClass1()
NewMethod('External call')(instance, "first", "second")
instance.method("first", "second")

External call: <__main__.MyClass1 object at 0x0000029FEDB8BD90> called with first and second
Internal call: <__main__.MyClass1 object at 0x0000029FEDB8BD90> called with first and second


### 메서드를 위한 빌트인 데코레이터
- @property, @classmethod, @staticmethod 데코레이터는 디스크립터
- 메서드를 인스턴스가 아닌 클래스에서 직접 호출할 때는 관습적으로 디스크립터 자체를 반환한다. 프로퍼티를 클래스에서 직접 호출하면 계산할 속성이 없으므로 일종의 디스크립터인 프로퍼티 객체 자체를 반환한다.
- @classmethod를 사용하면 디스크립터의 __get__ 함수가 메서드를 인스턴스에서 호출하든 클래스에서 호출하든 상관없이 데코레이팅 함수에 첫번째 파라미터로 메서드를 소유한 클래스를 넘겨준다.
- @staticmethod를 사용하면 정의한 파라미터 이외의 파라미터를 넘기지 않도록 한다. 즉 __get__메서드에서 첫번째 파라미터에 self를 바인딩하는 작업을 취소한다.

#### 슬롯(slot)

- 클래스에 __slot__ 속성을 정의하면 클래스가 기대하는 특정 속성만 정의하고 다른 것은 제한할 수 있다.
- `__slot__`에 정의되지 않은 속성을 동적으로 추가하려고 할 경우 AttributeError가 발생한다.    
이 경우 클래스는 `__dict__` 속성을 갖지 않는다.
- 이 때 객체의 사전이 없어도 속성을 가져올 수 있도록 디스크립터를 사용할 수 있다.
- 흥미로운 기능이지만, 파이썬의 동적인 특성을 없애기 때문에 신중히 사용하자.
- 메모리를 덜 사용한다는 점이 장점이다.

## 데코레이터를 디스크립터로 구현하기
- `__get__` 메서드를 구현하고 types.MethodType 을 사용해 데코레이터 자체를 객체에 바인딩된 메서드로 만드는 것
- 데코레이터를 객체로 구현해야함 
  - 함수로 구현하면 `__get__()` 메서드가 이미 존재해서 오류 발생할 수 있음

# 요약
- 파이썬의 경계를 metaprogramming에 가깝게 해주는 고급 기능
- 경락한 추상화를 통해 깔끔하고 컴팩트한 클래스를 만들 수 있음

- 오버 엔지니어링 주의 
  - 내부 API 개발, 라이브러리, 프레임워크 디자인 등에 권장
- 비즈니스 로직을 구현한 컴포넌트에서 사용하기 위한 기술적인 기능 구현만을 포함하고   
비즈니스 로직 자체를 포함하면 안됨
